In [ ]:
INPUT_CSV = "/home/ginasohn/step_tl/qwen_128_48_80gb_max4192_iter022_layer_010_mn_mk.csv"

: 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Read the CSV file
# Set your CSV filename here
df = pd.read_csv(INPUT_CSV)

# Extract the data (assuming the CSV has columns with these names)
# Create x-axis labels using tile_m and tile_n columns
labels = [f"{m}_{n}" for m, n in zip(df['tile_N'], df['tile_F'])]
on_chip_values = df['on_chip_requirement_bytes']
cycles_values = df['cycles']
off_chip_values = df['off_chip_traffic_bytes']

# Set up the chart with multiple y-axes
x = np.arange(len(labels))  # Label locations
width = 0.35  # Width of bars

fig, ax1 = plt.subplots(figsize=(14, 8))

# Create bars for off-chip metrics on primary y-axis
bars1 = ax1.bar(x + width/2, off_chip_values, width, label='Off-chip Traffic', alpha=0.8, color='orange')

# Customize the primary axis (bars)
ax1.set_xlabel('Tile Configuration ((batch_dim_tile)_(moe_inter_dim_tile))', fontsize=15)
ax1.set_ylabel('Off-chip Traffic', fontsize=15, color='black')
# ax1.set_title('Performance Metrics with Triple Y-Axes', fontsize=15, fontweight='bold')
ax1.set_xticks(x)
ax1.set_xticklabels(labels)
ax1.tick_params(axis='y', labelcolor='black')
ax1.grid(True, alpha=0.3, axis='y')

# Create secondary y-axis for cycles line chart
ax2 = ax1.twinx()
line2 = ax2.plot(x, cycles_values, color='red', marker='o', linewidth=2, markersize=8, label='Cycles')
ax2.set_ylabel('Cycles', fontsize=15, color='red')
ax2.tick_params(axis='y', labelcolor='red')
ax2.set_ylim(15000, 75000)

# Create third y-axis for on-chip requirement bars
ax3 = ax1.twinx()
# Offset the third y-axis to the right
ax3.spines['right'].set_position(('outward', 60))
bars2 = ax3.bar(x - width/2, on_chip_values, width, label='On-chip Requirement', alpha=0.8, color='steelblue')
ax3.set_ylabel('On-chip Requirement', fontsize=15, color='steelblue')
ax3.tick_params(axis='y', labelcolor='steelblue')
ax3.set_ylim(1e3, 7*1e6)

# Add value labels on bars
def add_value_labels_bars(bars, values, ax_ref):
    for bar, value in zip(bars, values):
        height = bar.get_height()
        ax_ref.annotate(f'{value:,.0f}' if abs(value) >= 1 else f'{value:.2f}',
                    xy=(bar.get_x() + bar.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom',
                    fontsize=9, fontweight='bold')

# Add value labels on bars
add_value_labels_bars(bars1, off_chip_values, ax1)
add_value_labels_bars(bars2, on_chip_values, ax3)
add_value_labels_line(x, cycles_values, ax2, color='red', offset=10)

# Combine legends from all axes
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
lines3, labels3 = ax3.get_legend_handles_labels()
ax1.legend(lines1 + lines2 + lines3, labels1 + labels2 + labels3,
           bbox_to_anchor=(0.5, -0.1), loc='upper center', ncol=3, fontsize=15)

# Adjust layout to prevent label cutoff
plt.tight_layout()

# Display the plot
plt.show()

# Optional: Save the plot
# plt.savefig('csv_triple_axis_chart.png', dpi=300, bbox_inches='tight')

# Print some info about the data
print(f"Data shape: {df.shape}")
print(f"Columns: {df.columns.tolist()}")
print(f"First few rows:")
print(df.head())
